In [1]:
#git@github.com:analyticsrepo01/gemini_video.git

In [2]:
!pip install -q --upgrade google-cloud-aiplatform
!pip install -q google-cloud-videointelligence
!pip install -q pytube
!pip install -q unidecode
!pip install -q varname
!pip install -q -U 'crewai[tools]' mdpdf 
!pip install -q langchain_google_vertexai
!pip install -q moviepy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 29.0.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 29.0.0 which is incompatible.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.1 which is incompatible.


In [3]:
# %pip install google-cloud-videointelligence
# %pip install unidecode
# %pip install varname
%pip install seaborn
!sudo apt-get -q install -y ffmpeg

Note: you may need to restart the kernel to use updated packages.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  i965-va-driver intel-media-va-driver libaacs0 libaom0 libass9 libasyncns0
  libavc1394-0 libavcodec58 libavdevice58 libavfilter7 libavformat58
  libavresample4 libavutil56 libbdplus0 libbluray2 libbs2b0 libcaca0
  libcdio-cdda2 libcdio-paranoia2 libcdio19 libchromaprint1 libcodec2-0.9
  libdav1d4 libdc1394-25 libfftw3-double3 libflac8 libflite1 libgbm1 libgme0
  libgsm1 libiec61883-0 libigdgmm11 libjack-jackd2-0 liblilv-0-0 libmfx1
  libmp3lame0 libmpg123-0 libmysofa1 libnorm1 libogg0 libopenal-data
  libopenal1 libopenjp2-7 libopenmpt0 libopus0 libpgm-5.3-0 libpocketsphinx3
  libpostproc55 libpulse0 librabbitmq4 libraw1394-11 librubberband2
  libsamplerate0 libsdl2-2.0-0 libserd-0-0 libshine3 libsndfile1 libsndio7.0
  libsodium23 libsord-0-0 libsoxr0 libspeex1 libsphinxbase3 libsratom-

In [4]:
"""Transcribe speech from a video stored on GCS."""
from google.cloud import videointelligence
from pytube import YouTube
# Install pytube if not already installed
from pytube import YouTube
from google.cloud import storage
import vertexai
import json
import unidecode
from varname import nameof

import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.preview.language_models import (ChatModel, InputOutputTextPair,
                                              TextEmbeddingModel,
                                              TextGenerationModel)
from vertexai.generative_models import GenerativeModel

import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason

### 1.1) Setup

In [5]:
import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}
! mkdir output

Updated property [core/project].
Creating gs://mythical-lens-406709-renzinstance3-asia-southeast1/...


In [6]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    # model_name="gemini-pro", # Replace with your desired Gemini model
    # model = GenerativeModel("gemini-pro")
    # model = GenerativeModel("gemini-1.5-pro-preview-0215")
    model = "gemini-1.5-pro-preview-0409",
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


def generate_pro(input_prompt):
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    full_prompt = '''summarize the prompt below and do note prompt below will be send to imagen mode so please clean up any sensitve words and replace them into unblocked words like replace girl or lady can be replaced by female human and so on : ''' + input_prompt
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 8190,
        "temperature": 0.2,
        "top_p": 1
    },stream=False,)
    
    # print (responses.text)
    
    return(responses.text)



In [7]:
def generate(input_array, max_output_tokens=8192, temperature=1, top_p=0.95):
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    params = {}
    params["safety_settings"] = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }
    params["generation_config"] = {"max_output_tokens": max_output_tokens,"temperature": temperature,"top_p": top_p}
    params["stream"] = True
    ##responses = model.generate_content(input_array,generation_config=generation_config,safety_settings=safety_settings, stream=True)
    responses = model.generate_content(input_array,**params)
    return responses

In [8]:
from google.cloud import storage
# -1) Helper functions for bucket management
# -1.1) Fetching from youtube
def download_video(video_url):
    # Create a YouTube object
    yt = YouTube(video_url)
    
    # Filter to get the highest resolution stream that includes both video and audio
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    
    # Download the video
    if stream:
        print(f"Going to download {video_url}")
        stream.download()
        print(f"Downloaded into {stream.default_filename}")
    else:
        print("No suitable stream found")
    return stream.default_filename
def upload_video_sur_bucket(video_url):
    downloaded_video     = download_video(video_url)
    uploaded_video       = upload_file_to_bucket(bucket_name, downloaded_video)
    
    uri_video_sur_bucket = f"gs://{bucket_name}/{uploaded_video}"

    return uri_video_sur_bucket
# -1.2) Managing buckets
def list_buckets_names():
    storage_client = storage.Client()
    buckets        = storage_client.list_buckets()
    return [bucket.name for bucket in buckets]


def create_bucket_class_location(bucket_name, storage_class = "STANDARD", location="us"):
    storage_client       = storage.Client()
    bucket               = storage_client.bucket(bucket_name)
    bucket.storage_class = storage_class
    new_bucket           = storage_client.create_bucket(bucket, location=location)
    print(f"Created bucket {new_bucket.name} in {new_bucket.location} with storage class {new_bucket.storage_class}")
    return new_bucket


def upload_file_to_gcs(bucket_name, source_file_path, destination_blob_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_path)
    return blob
def upload_file_to_bucket(bucket_name, source_file_path_name): 
    source_file_path = f"./{source_file_path_name}"
    destination_blob_name = source_file_path_name.replace(' ', '_')
    destination_blob_name = unidecode.unidecode(destination_blob_name)
    uploaded_blob         = upload_file_to_gcs(bucket_name, source_file_path, destination_blob_name)
    os.rename(f'{source_file_path_name}', f'{destination_blob_name}')
    print(f"File [{source_file_path}] \nUploaded to [{uploaded_blob.name}] \nIn bucket [{bucket_name}]")
    return destination_blob_name
def create_bucket_if_doesnt_exist(bucket_name):
    if bucket_name not in list_buckets_names():
        print(f"Going to create bucket :{bucket_name}")
        create_bucket_class_location(bucket_name)
        
    assert(bucket_name in list_buckets_names())

    
# - 1.3) 🔥 Calling Gemini 🔥 (the magic is here ! )
def generate(input_array, max_output_tokens=8192, temperature=1, top_p=0.95):
    model = GenerativeModel("gemini-1.5-pro-preview-0409")
    params = {}
    params["safety_settings"] = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }
    params["generation_config"] = {"max_output_tokens": max_output_tokens,"temperature": temperature,"top_p": top_p}
    params["stream"] = True
    ##responses = model.generate_content(input_array,generation_config=generation_config,safety_settings=safety_settings, stream=True)
    responses = model.generate_content(input_array,**params)
    return responses

In [9]:
# Define the video URL
video_url = "https://www.youtube.com/watch?v=V6DJYGn2SFk"

In [10]:
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [11]:
# 1.2) transfer youtube video to the bucket

downloaded_video = download_video(video_url)
uploaded_video = upload_file_to_bucket(BUCKET_NAME, downloaded_video)

Going to download https://www.youtube.com/watch?v=V6DJYGn2SFk
Downloaded into Google Cloud Next 24 Opening Keynote.mp4
File [./Google Cloud Next 24 Opening Keynote.mp4] 
Uploaded to [Google_Cloud_Next_24_Opening_Keynote.mp4] 
In bucket [mythical-lens-406709-renzinstance3-asia-southeast1]


In [12]:
from vertexai.generative_models import GenerativeModel

import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models


In [13]:
import os

filename = uploaded_video
base_name = os.path.basename(filename)  # Get the base filename
name_without_ext = os.path.splitext(base_name)[0]  # Split off the extension 

print(base_name) 

Google_Cloud_Next_24_Opening_Keynote.mp4


### 1.2) Split the video into chunks for batch processing

In [14]:
! ffmpeg -i {base_name} -c copy -map 0 -f segment -segment_time 300 -reset_timestamps 1 output/{name_without_ext}_%02d.mp4

ffmpeg version 4.3.6-0+deb11u1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab 

In [15]:
directory = 'output/'

def get_creation_time(item):
        item_path = os.path.join(directory, item)
        return os.path.getctime(item_path)

entries = os.listdir(directory)
sorted_items = sorted(entries, key=get_creation_time)
videos = []
for video in sorted_items:
    upload_file_to_gcs(BUCKET_NAME, f"{directory}{video}", f"{directory}{video}")

In [16]:
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
blob_names = [blob.name for blob in bucket.list_blobs(prefix = "output/")]

videos=[]
for blob in blob_names:
    uri_video_sur_bucket = f"gs://{BUCKET_NAME}/{blob}"
    mime_type = "video/mp4"
    parameters = {"mime_type"  : mime_type, 
                  "uri"        : uri_video_sur_bucket }
    videos.append(Part.from_uri(**parameters))

### 1.3) Add the video as a Part of the model

In [17]:
response_list = []
for video in videos:
    prompt = """"Create a chapter index for this video and generate a nice HTML code to present the chapter index."""
    responses = generate([prompt, video])
    response_list.append(responses)

In [18]:
result = []
for responses in response_list:
    for response in responses :
        print(".",end="")
        result.append(response.text)

....................................................................................................................................................................................................................................

In [19]:
print("\n".join(result))


## Google
 Cloud Next '23 Opening Keynote Chapter Index

While I don't have access to the specific content of the video, I can create a general chapter index
 based on common themes in such keynotes and the provided transcript excerpt:

**HTML Code:**

```html
<!DOCTYPE html>
<html>

<head>
<title>Google Cloud Next '23 Opening Keynote - Chapter Index</title>
<style>
body { font-family: sans-serif; }
h2 { color: #3367d6; } /* Google blue */
ul { list-style: none
; padding: 0; }
li a { display: block; padding: 8px; text-decoration: none; color: #000; }
li a:hover { background-color: #eee; }
</style>
</head>
<body>
<h1>Google
 Cloud Next '23 Opening Keynote - Chapter Index</h1>

<h2>Introduction (00:00 - 01:43)</h2>
<ul>
  <li><a href="#intro">Opening and Welcome</a></li>
  <li><a href="#ai-today">AI as a Today Thing - Examples of Impact</a></li>
</ul>

<h2>Thomas Kurian Keynote (01:43 - 0
3:34)</h2>
<ul>
  <li><a href="#progress">Google Cloud Progress and Achievements</a></li>
  <li><a h